In [ ]:
!wget https://mind201910small.blob.core.windows.net/release/MINDlarge_train.zip
!wget https://mind201910small.blob.core.windows.net/release/MINDlarge_dev.zip    
!mkdir train
!mkdir val
!unzip MINDlarge_train.zip -d ./train
!unzip MINDlarge_dev.zip -d ./val

In [ ]:
import csv
import random
import json
import numpy as np
from numpy.linalg import cholesky
from keras.utils.np_utils import to_categorical
from keras.layers import *
from keras.models import Model, load_model 
from keras import activations, constraints, initializers, regularizers
from keras import backend as K 
from keras.engine.topology import Layer, InputSpec
from keras import initializers #keras2
from sklearn.metrics import *
from keras.optimizers import *
import keras 
from nltk.tokenize import word_tokenize 

In [ ]:
with open('train/behaviors.tsv')as f:
    trainuser=f.readlines()

with open('val/behaviors.tsv') as f:
    valuser=f.readlines()

with open('train/news.tsv')as f:
    data=f.readlines()

with open('val/news.tsv')as f:
    data+=f.readlines()

with open('train/entity_embedding.vec')as f:
    entity_emb=f.readlines()
    
with open('val/entity_embedding.vec')as f:
    entity_emb+=f.readlines()

entity_emb_dict={}
for i in entity_emb:
    entity_emb_dict[i.strip().split('\t')[0]]=[float(x) for x in i.strip().split('\t')[1:]]

entityidlist={'NULL':0}
entity_emb_table=[[0.]*100]
cnt=0.
for i in news:
    for j in news[i][3]:
        if j not in entityidlist:
            entityidlist[j]=len(entityidlist)
            entity_emb_table.append(entity_emb_dict.get(j,np.random.uniform(-0.03,0.03,(100,))))
entity_emb_table=np.array(entity_emb_table,dtype=np.float32)
print(entity_emb_table.shape)


news={}
category={'NULL':0}
subcategory={'NULL':0}
newsnumber=0
for i in data:
    line=i.strip('\n').split('\t')
    if line[0] not in news:
        news[line[0]]=[line[1],line[2],word_tokenize(line[3].lower()),[x["WikidataId"] for x in json.loads(line[6])]]
    if line[1] not in category:
        category[line[1]]=len(category)
    if line[2] not in subcategory:
        subcategory[line[2]]=len(subcategory)
    newsnumber+=1
    if newsnumber%1000==0:
        print(newsnumber)

newsindex={'NULL':0}
for i in news:
    newsindex[i]=len(newsindex)

word_dict={'PADDING':[0,999999]}

for i in news:
    for j in news[i][2]:
        if j in word_dict:
            word_dict[j][1]+=1
        else:
            word_dict[j]=[len(word_dict),1]

In [ ]:
!wget https://nlp.stanford.edu/data/glove.840B.300d.zip
!unzip glove.840B.300d.zip

In [ ]:
            
embdict={}
with open('glove.840B.300d.txt','rb')as f:
    linenb=0
    while True:
        line=f.readline()
        if len(line)==0:
            break
        line = line.split()
        word=line[0].decode()
        linenb+=1
        if len(word) != 0:
            vec=[float(x) for x in line[1:]]
            if word in word_dict:
                embdict[word]=vec


emb_mat=[0]*len(word_dict)
in_dict_emb=[]
for i in embdict.keys():
    emb_mat[word_dict[i][0]]=np.array(embdict[i],dtype='float32')
    in_dict_emb.append(emb_mat[word_dict[i][0]])
in_dict_emb=np.array(in_dict_emb,dtype='float32')

mu=np.mean(in_dict_emb, axis=0)
Sigma=np.cov(in_dict_emb.T)

norm=np.random.multivariate_normal(mu, Sigma, 1)
print(mu.shape,Sigma.shape,norm.shape)

for i in range(len(emb_mat)):
    if type(emb_mat[i])==int:
        emb_mat[i]=np.reshape(norm, 300)
emb_mat[0]=np.zeros(300,dtype='float32')
emb_mat=np.array(emb_mat,dtype='float32')
print(emb_mat.shape)

In [ ]:
news_words=[[0]*30]

for i in news:
    words=[]
    for word in news[i][2]:
        if word in word_dict:
            words.append(word_dict[word][0])
    words=words[:30]
    news_words.append(words+[0]*(30-len(words)))
news_words=np.array(news_words,dtype='int32') 

news_entity=[[0]*5]

for i in news:
    entities=[]
    for entity in news[i][3]:
        if entity in entityidlist:
            entities.append(entityidlist[entity])
    entities=entities[:5]
    entities=entities+[0]*(5-len(entities))
    news_entity.append(entities)
news_entity=np.array(news_entity,dtype='int32') 

news_topic=[0]
for i in news:
     
    news_topic.append(category[news[i][0]])
news_topic=np.array(news_topic,dtype='int32')      

In [ ]:
def newsample(array,ratio):
    if ratio >len(array):
        return random.sample(array*(ratio//len(array)+1),ratio)
    else:
        return random.sample(array,ratio)
    
npratio=4
train_candidate=[]    
train_label=[]
train_user_his=[]

for user in trainuser:
    userline=user.replace('\n','').split('\t')
    clickids=[newsindex[x] for x in userline[3].split()][-50:]
    pdoc=[newsindex[x.split('-')[0]] for x in userline[4].split() if x.split('-')[1]=='1']
    ndoc=[newsindex[x.split('-')[0]] for x in userline[4].split() if x.split('-')[1]=='0']
    
    for doc in pdoc:
        negd=newsample(ndoc,npratio)
        negd.append(doc)
        candidate_label=[0]*npratio+[1]
        candidate_order=list(range(npratio+1))
        random.shuffle(candidate_order)
        candidate_shuffle=[]
        candidate_label_shuffle=[]
        for i in candidate_order:
            candidate_shuffle.append(negd[i])
            candidate_label_shuffle.append(candidate_label[i])
        train_candidate.append(candidate_shuffle)
        train_label.append(candidate_label_shuffle)
        train_user_his.append(clickids+[0]*(50-len(clickids))) 


# In[33]:





test_candidate=[] 
test_user_his=[]
test_index=[]
test_label=[]

for user in valuser:
    userline=user.replace('\n','').split('\t')
    clickids=[newsindex[x] for x in userline[3].split()][-50:]
    docs=[newsindex[x.split('-')[0]] for x in userline[4].split()]
    index=[]
    index.append(len(test_candidate))
  
    test_user_his.append(clickids+[0]*(50-len(clickids)))
    for x in userline[4].split():
        test_label.append(int(x.split('-')[1]))
    for doc in docs:
        test_candidate.append(doc)
    index.append(len(test_candidate))
    test_index.append(index)


train_candidate=np.array(train_candidate,dtype='int32')
train_label=np.array(train_label,dtype='int32')
train_user_his=np.array(train_user_his,dtype='int32')

test_candidate=np.array(test_candidate,dtype='int32') 
test_user_his=np.array(test_user_his,dtype='int32')
test_label=np.array(test_label,dtype='int32')

In [ ]:
def generate_batch_data_random(batch_size):
    idx = np.arange(len(train_label))
    np.random.shuffle(idx)
    y=train_label
    batches = [idx[range(batch_size*i, min(len(y), batch_size*(i+1)))] for i in range(len(y)//batch_size+1)]
 
    while (True):
        for i in batches:
            
            item_words = news_words[train_candidate[i]]
            
            item_topic=news_topic[train_candidate[i]]
            item_entity=news_entity[train_candidate[i]]
            
            user_his=news_words[train_user_his[i]]
            user_topic=news_topic[train_user_his[i]]
            user_entity=news_entity[train_user_his[i]]
            user_entity_feature=[]
            all_A=[]
            for s in range(len(i)):
                Asize=len(train_user_his[i][s])+len(category)+50
                newsA=np.zeros((Asize,Asize))
                entityid_set={}
                entityid_set_ids=[]
                for el in range(len(user_entity[s])):
                    for e in range(len(user_entity[s][el])):
                        if user_entity[s][el][e] not in entityid_set:
                            entityid_set[user_entity[s][el][e]]=len(entityid_set)
                            entityid_set_ids.append(user_entity[s][el][e])
                        if user_entity[s][el][e]!=0  and entityid_set[user_entity[s][el][e]]<50:
                            newsA[el][len(train_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]]=1
                            newsA[len(train_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]][el]=1
                            newsA[len(train_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]][len(train_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]]=1
                entityid_set_ids=entityid_set_ids[:50]
                entityid_set_ids+=[0]*(50-len(entityid_set_ids))
        
                for m in range(len(train_user_his[i][s])):
                    if train_user_his[i][s][m]!=0:
                        newsA[m][m]=1
                        if m>=1 and train_user_his[i][s][m-1]!=0:
                            newsA[m][m-1]=1
                        if m<len(train_user_his[i][s])-1 and train_user_his[i][s][m+1]!=0:
                            newsA[m][m+1]=1
                    if user_topic[s][m]!=0:
                        newsA[m][len(train_user_his[i][s])+user_topic[s][m]]=1
                        newsA[len(train_user_his[i][s])+user_topic[s][m]][m]=1
                        newsA[len(train_user_his[i][s])+user_topic[s][m]][len(train_user_his[i][s])+user_topic[s][m]]=1
                user_entity_feature.append(entityid_set_ids)
                all_A.append(newsA)
            user_entity_feature=np.array(user_entity_feature)                
    
    
            all_A=np.array(all_A,dtype='float32')
            
            topics=np.array([list(range(len(category)))]*len(i),dtype='int32')
                   
            yield ([item_words,user_his,item_topic,item_entity,user_entity_feature,topics,all_A], [train_label[i]])

In [ ]:
def dcg_score(y_true, y_score, k=10):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])
    gains = 2 ** y_true - 1
    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gains / discounts)


def ndcg_score(y_true, y_score, k=10):
    best = dcg_score(y_true, y_true, k)
    actual = dcg_score(y_true, y_score, k)
    return actual / best


def mrr_score(y_true, y_score):
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order)
    rr_score = y_true / (np.arange(len(y_true)) + 1)
    return np.sum(rr_score) / np.sum(y_true)

In [ ]:
#Taken from the GAT repo
class GraphAttention(Layer):

    def __init__(self,
                 F_,
                 attn_heads=20,
                 attn_heads_reduction='concat',  # {'concat', 'average'}
                 dropout_rate=0.2,
                 activation='relu',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 bias_initializer='zeros',
                 attn_kernel_initializer='glorot_uniform',
                 kernel_regularizer=None,
                 bias_regularizer=None,
                 attn_kernel_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 bias_constraint=None,
                 attn_kernel_constraint=None,
                 **kwargs):
        if attn_heads_reduction not in {'concat', 'average'}:
            raise ValueError('Possbile reduction methods: concat, average')

        self.F_ = F_  # Number of output features (F' in the paper)
        self.attn_heads = attn_heads  # Number of attention heads (K in the paper)
        self.attn_heads_reduction = attn_heads_reduction  # Eq. 5 and 6 in the paper
        self.dropout_rate = dropout_rate  # Internal dropout rate
        self.activation = activations.get(activation)  # Eq. 4 in the paper
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.attn_kernel_initializer = initializers.get(attn_kernel_initializer)

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)
        self.attn_kernel_regularizer = regularizers.get(attn_kernel_regularizer)
        self.activity_regularizer = regularizers.get(activity_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.bias_constraint = constraints.get(bias_constraint)
        self.attn_kernel_constraint = constraints.get(attn_kernel_constraint)
        self.supports_masking = False

        # Populated by build()
        self.kernels = []       # Layer kernels for attention heads
        self.biases = []        # Layer biases for attention heads
        self.attn_kernels = []  # Attention kernels for attention heads

        if attn_heads_reduction == 'concat':
            # Output will have shape (..., K * F')
            self.output_dim = self.F_ * self.attn_heads
        else:
            # Output will have shape (..., F')
            self.output_dim = self.F_

        super(GraphAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) >= 2
        F = input_shape[0][-1]

        # Initialize weights for each attention head
        for head in range(self.attn_heads):
            # Layer kernel
            kernel = self.add_weight(shape=(F, self.F_),
                                     initializer=self.kernel_initializer,
                                     regularizer=self.kernel_regularizer,
                                     constraint=self.kernel_constraint,
                                     name='kernel_{}'.format(head))
            self.kernels.append(kernel)

            # # Layer bias
            if self.use_bias:
                bias = self.add_weight(shape=(self.F_, ),
                                       initializer=self.bias_initializer,
                                       regularizer=self.bias_regularizer,
                                       constraint=self.bias_constraint,
                                       name='bias_{}'.format(head))
                self.biases.append(bias)

            # Attention kernels
            attn_kernel_self = self.add_weight(shape=(self.F_, 1),
                                               initializer=self.attn_kernel_initializer,
                                               regularizer=self.attn_kernel_regularizer,
                                               constraint=self.attn_kernel_constraint,
                                               name='attn_kernel_self_{}'.format(head),)
            attn_kernel_neighs = self.add_weight(shape=(self.F_, 1),
                                                 initializer=self.attn_kernel_initializer,
                                                 regularizer=self.attn_kernel_regularizer,
                                                 constraint=self.attn_kernel_constraint,
                                                 name='attn_kernel_neigh_{}'.format(head))
            self.attn_kernels.append([attn_kernel_self, attn_kernel_neighs])
        self.built = True

    def call(self, inputs):
        X = inputs[0]  # Node features (N x F)
        A = inputs[1]  # Adjacency matrix (N x N)

        outputs = []
        for head in range(self.attn_heads):
            kernel = self.kernels[head]  # W in the paper (F x F')
            attention_kernel = self.attn_kernels[head]  # Attention kernel a in the paper (2F' x 1)

            # Compute inputs to attention network
            features = K.dot(X, kernel)  # (N x F')

            # Compute feature combinations
            # Note: [[a_1], [a_2]]^T [[Wh_i], [Wh_2]] = [a_1]^T [Wh_i] + [a_2]^T [Wh_j]
            attn_for_self = K.dot(features, attention_kernel[0])    # (N x 1), [a_1]^T [Wh_i]
            attn_for_neighs = K.dot(features, attention_kernel[1])  # (N x 1), [a_2]^T [Wh_j]
            # Attention head a(Wh_i, Wh_j) = a^T [[Wh_i], [Wh_j]]
            dense = attn_for_self + K.permute_dimensions(attn_for_neighs,(0,2,1))  # (N x N) via broadcasting

            # Add nonlinearty
            dense = LeakyReLU(alpha=0.2)(dense)

            # Mask values before activation (Vaswani et al., 2017)
            mask = -10e9 * (1.0 - A)
            dense += mask

            # Apply softmax to get attention coefficients
            dense = K.softmax(dense)  # (N x N)

            # Apply dropout to features and attention coefficients
            dropout_attn = Dropout(self.dropout_rate)(dense)  # (N x N)
            dropout_feat = Dropout(self.dropout_rate)(features)  # (N x F')

            # Linear combination with neighbors' features
            node_features = K.batch_dot(dropout_attn, dropout_feat,axes=[2,1])  # (N x F')
            
            if self.use_bias:
                node_features = K.bias_add(node_features, self.biases[head])
            
            # Add output of attention head to final output
            outputs.append(node_features)

        # Aggregate the heads' output according to the reduction method
        if self.attn_heads_reduction == 'concat':
            output = K.concatenate(outputs)  # (N x KF')
        else:
            output = K.mean(K.stack(outputs), axis=0)  # N x F')
        
        output = self.activation(output)
        return output

    def compute_output_shape(self, input_shape):
        output_shape = (input_shape[0][0], input_shape[0][1], self.output_dim)
        return output_shape

In [ ]:
class Attention(Layer):

    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ', 
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK', 
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV', 
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
        
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
                
    def call(self, x): 
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x 
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3)) 
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))    
        A = K.softmax(A) 
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
        
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [ ]:
class AttLayer(Layer):
    def __init__(self, dim=200,**kwargs):
        self.init = initializers.get('normal')
        self.dim = dim
        super(AttLayer, self).__init__(**kwargs)
    def build(self, input_shape):
        dim = self.dim
        self.W1 = K.variable(self.init((input_shape[-1], dim)))
        self.b1 = K.variable(self.init((dim,)))
        self.q1 = K.variable(self.init((dim, 1)))
         
        self.trainable_weights = [self.W1, self.b1, self.q1]
        super(AttLayer,self).build(input_shape)  

    def call(self, inputs, **kwargs): 
        attention1 = K.tanh(K.dot(inputs, self.W1) + self.b1)
        attention1 = K.dot(attention1, self.q1)
        attention1 = K.squeeze(attention1, axis=2)
        
        attention = attention1
        attention = K.exp(attention)
        attention_weight = attention / (K.sum(attention, axis=-1, keepdims=True) + K.epsilon())

        attention_weight = K.expand_dims(attention_weight)
        weighted_input = inputs * attention_weight
        return K.sum(weighted_input, axis=1)

    def compute_mask(self, input, input_mask=None):
        return None

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

In [ ]:
MAX_SENT_LENGTH=30
MAX_SENTS=50
keras.backend.clear_session() 
sentence_input = Input(shape=(MAX_SENT_LENGTH,), dtype='int32')
topic_input = Input(shape=(1,), dtype='int32')

embedding_layer = Embedding(len(word_dict), 300, weights=[emb_mat],trainable=True)

topic_embedding_layer = Embedding(len(category),256,trainable=True)
entity_embedding_layer = Embedding(len(entity_emb_table),100, weights=[entity_emb_table],trainable=True)

embedded_sequences = embedding_layer(sentence_input)

embedded_sequences=Dropout(0.2)(embedded_sequences)
embedded_sequences=Dropout(0.2)(Attention(20,20)([embedded_sequences,embedded_sequences,embedded_sequences]))

textrep=Dense(256,activation='relu')(AttLayer()(embedded_sequences))

text_encoder = Model([sentence_input], textrep)

review_input = Input((MAX_SENTS,MAX_SENT_LENGTH,), dtype='int32')
review_encoders= TimeDistributed(text_encoder)(review_input)

review_encoders = Dropout(0.2)(review_encoders)
A_input = Input(shape=(MAX_SENTS+len(category)+50,MAX_SENTS+len(category)+50), dtype='float32')

all_topic_input = Input((len(category),), dtype='int32')
all_entity_input = Input((50,), dtype='int32')

all_topic_emb=topic_embedding_layer(all_topic_input)
all_entity_emb=Dense(256)(entity_embedding_layer(all_entity_input))
xinput=concatenate([review_encoders ,all_topic_emb, all_entity_emb],axis=1)

graph_attention_basis = GraphAttention(16,
                           attn_heads=16,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([xinput, A_input])


graph_pool_n1 = GraphAttention(16,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([graph_attention_basis, A_input])
graph_pool_n1 =Activation('softmax')(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(Dense(50)(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(graph_pool_n1))))
graph_pool_n1=concatenate([graph_pool_n1,Lambda(lambda x:K.zeros_like(x[:,:,:16]))(all_topic_emb),Lambda(lambda x:K.zeros_like(x[:,:,:16]))(all_entity_emb)],axis=1)


graph_pool_t1 = GraphAttention(3,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([graph_attention_basis, A_input])
graph_pool_t1 =Activation('softmax')(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(Dense(len(category))(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(graph_pool_t1))))
graph_pool_t1=concatenate([Lambda(lambda x:K.zeros_like(x[:,:,:3]))(review_encoders),graph_pool_t1,Lambda(lambda x:K.zeros_like(x[:,:,:3]))(all_entity_emb)],axis=1)


graph_pool_e1 = GraphAttention(9,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([graph_attention_basis, A_input])
graph_pool_e1 =Activation('softmax')(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(Dense(50)(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(graph_pool_e1))))
graph_pool_e1=concatenate([Lambda(lambda x:K.zeros_like(x[:,:,:9]))(review_encoders),Lambda(lambda x:K.zeros_like(x[:,:,:9]))(all_topic_emb),graph_pool_e1],axis=1)


Apool_nn1=Dot((2,1))([Permute((2,1))(graph_pool_n1),Dot((1,1))([A_input,graph_pool_n1])])
xpool_n1=Dot((1,1))([graph_pool_n1,graph_attention_basis])

Apool_tt1=Dot((2,1))([Permute((2,1))(graph_pool_t1),Dot((1,1))([A_input,graph_pool_t1])])
xpool_t1=Dot((1,1))([graph_pool_t1,graph_attention_basis])

Apool_ee1=Dot((2,1))([Permute((2,1))(graph_pool_e1),Dot((1,1))([A_input,graph_pool_e1])])
xpool_e1=Dot((1,1))([graph_pool_e1,graph_attention_basis])

Apool_nt1=Dot((2,1))([Permute((2,1))(graph_pool_n1),Dot((1,1))([A_input,graph_pool_t1])])
Apool_tn1=Dot((2,1))([Permute((2,1))(graph_pool_t1),Dot((1,1))([A_input,graph_pool_n1])])
Apool_ne1=Dot((2,1))([Permute((2,1))(graph_pool_n1),Dot((1,1))([A_input,graph_pool_e1])])
Apool_en1=Dot((2,1))([Permute((2,1))(graph_pool_e1),Dot((1,1))([A_input,graph_pool_n1])])
Apool_te1=Dot((2,1))([Permute((2,1))(graph_pool_t1),Dot((1,1))([A_input,graph_pool_e1])])
Apool_et1=Dot((2,1))([Permute((2,1))(graph_pool_e1),Dot((1,1))([A_input,graph_pool_t1])])
Ap1=concatenate([concatenate([Apool_nn1,Apool_tn1,Apool_en1],axis=1),concatenate([Apool_nt1,Apool_tt1,Apool_et1],axis=1),concatenate([Apool_ne1,Apool_te1,Apool_ee1],axis=1)])
xp1=concatenate([xpool_n1,xpool_t1,xpool_e1],axis=1)

graph_pool_n2 = GraphAttention(1,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([xp1, Ap1])
graph_pool_n2 =Activation('softmax')(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(Dense(16)(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(graph_pool_n2))))
graph_pool_n2=concatenate([graph_pool_n2,Lambda(lambda x:K.zeros_like(x[:,:,:1]))(xpool_t1),Lambda(lambda x:K.zeros_like(x[:,:,:1]))(xpool_e1)],axis=1)

graph_pool_t2 = GraphAttention(1,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([xp1, Ap1])
graph_pool_t2 =Activation('softmax')(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(Dense(3)(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(graph_pool_t2))))
graph_pool_t2=concatenate([Lambda(lambda x:K.zeros_like(x[:,:,:1]))(xpool_n1),graph_pool_t2,Lambda(lambda x:K.zeros_like(x[:,:,:1]))(xpool_e1)],axis=1)

graph_pool_e2 = GraphAttention(1,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([xp1, Ap1])
graph_pool_e2 =Activation('softmax')(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(Dense(9)(Lambda(lambda x:K.permute_dimensions(x,(0,2,1)))(graph_pool_e2))))
graph_pool_e2=concatenate([Lambda(lambda x:K.zeros_like(x[:,:,:1]))(xpool_n1),Lambda(lambda x:K.zeros_like(x[:,:,:1]))(xpool_t1),graph_pool_e2],axis=1)


Apool_nn2=Dot((2,1))([Permute((2,1))(graph_pool_n2),Dot((1,1))([Ap1,graph_pool_n2])])
xpool_n2=Dot((1,1))([graph_pool_n2,xp1])

Apool_tt2=Dot((2,1))([Permute((2,1))(graph_pool_t2),Dot((1,1))([Ap1,graph_pool_t2])])
xpool_t2=Dot((1,1))([graph_pool_t2,xp1])

Apool_ee2=Dot((2,1))([Permute((2,1))(graph_pool_e2),Dot((1,1))([Ap1,graph_pool_e2])])
xpool_e2=Dot((1,1))([graph_pool_e2,xp1])

Apool_nt2=Dot((2,1))([Permute((2,1))(graph_pool_n2),Dot((1,1))([Ap1,graph_pool_t2])])
Apool_tn2=Dot((2,1))([Permute((2,1))(graph_pool_t2),Dot((1,1))([Ap1,graph_pool_n2])])
Apool_ne2=Dot((2,1))([Permute((2,1))(graph_pool_n2),Dot((1,1))([Ap1,graph_pool_e2])])
Apool_en2=Dot((2,1))([Permute((2,1))(graph_pool_e2),Dot((1,1))([Ap1,graph_pool_n2])])
Apool_te2=Dot((2,1))([Permute((2,1))(graph_pool_t2),Dot((1,1))([Ap1,graph_pool_e2])])
Apool_et2=Dot((2,1))([Permute((2,1))(graph_pool_e2),Dot((1,1))([Ap1,graph_pool_t2])])
Ap2=concatenate([concatenate([Apool_nn2,Apool_tn2,Apool_en2],axis=1),concatenate([Apool_nt2,Apool_tt2,Apool_et2],axis=1),concatenate([Apool_ne2,Apool_te2,Apool_ee2],axis=1)])
xp2=concatenate([xpool_n2,xpool_t2,xpool_e2],axis=1)

diff_pool = GraphAttention(1,
                           attn_heads=1,
                           attn_heads_reduction='concat',
                           dropout_rate=0.2,
                           activation='elu')([xp2, Ap2])

diff_pool =Flatten()(Activation('softmax')(diff_pool))
u_att=keras.layers.Dot((1, 1))([xp2, diff_pool])


candidates=Input((1+npratio,MAX_SENT_LENGTH,), dtype='int32')
candidate_vecs=TimeDistributed(text_encoder)(candidates)

candidatestopic=Input((1+npratio,), dtype='int32')
candidatesentity=Input((1+npratio,5), dtype='int32')
candidate_topicemb=topic_embedding_layer(candidatestopic)
entity_dim_dense=Dense(256)
entity_att=AttLayer()
candidate_entityemb = TimeDistributed(entity_att)(TimeDistributed(TimeDistributed(entity_dim_dense))(entity_embedding_layer(candidatesentity)))

view_emb=concatenate([Lambda(lambda y:K.expand_dims(y,axis=2))(x) for x in [candidate_vecs,candidate_topicemb,candidate_entityemb]],axis=2)

view_att=AttLayer()

candidate_emb=TimeDistributed(view_att)(view_emb)

logits = Lambda(lambda x:K.clip(x,-5,5))(keras.layers.dot([u_att, candidate_emb], axes=-1))
logits = keras.layers.Activation(keras.activations.softmax)(logits)


model = Model([candidates,review_input,candidatestopic,candidatesentity,all_entity_input,all_topic_input ,A_input], [logits])
model.compile(loss=['categorical_crossentropy'], optimizer=Adam(lr=0.0001), metrics=['acc'])


candidate_one = keras.Input((MAX_SENT_LENGTH,))
candidate_one_topic_input = Input(shape=(1,), dtype='int32')
candidate_one_entity_input = Input(shape=(5,), dtype='int32')
candidate_one_vec = text_encoder([candidate_one])
candidate_one_topicemb=topic_embedding_layer(candidate_one_topic_input)
candidate_one_entityemb=entity_att(TimeDistributed(entity_dim_dense)(entity_embedding_layer(candidate_one_entity_input)))

candidate_one_view_emb=concatenate([Lambda(lambda y:K.expand_dims(y,axis=1))(x) for x in [candidate_one_vec,candidate_one_entityemb]]+[candidate_one_topicemb],axis=1)

candidate_one_vec = view_att(candidate_one_view_emb)
candidate_encoder = keras.Model([candidate_one,candidate_one_topic_input,candidate_one_entity_input], candidate_one_vec)

In [ ]:
def generate_batch_data_user(batch_size):
    idlist = np.arange(len(test_user_his))  
    batches = [idlist[range(batch_size*i, min(len(test_user_his), batch_size*(i+1)))] for i in range(len(test_user_his)//batch_size+1)]
    while (True):
        for i in batches: 

            user_his=news_words[test_user_his[i]]
            user_topic=news_topic[test_user_his[i]]
            user_entity=news_entity[test_user_his[i]]
            user_entity_feature=[]
            all_A=[]
            for s in range(len(i)):
                Asize=len(test_user_his[i][s])+len(category)+50
                newsA=np.zeros((Asize,Asize))
                entityid_set={}
                entityid_set_ids=[]
                for el in range(len(user_entity[s])):
                    for e in range(len(user_entity[s][el])):
                        if user_entity[s][el][e] not in entityid_set:
                            entityid_set[user_entity[s][el][e]]=len(entityid_set)
                            entityid_set_ids.append(user_entity[s][el][e])
                        if user_entity[s][el][e]!=0  and entityid_set[user_entity[s][el][e]]<50:
                            newsA[el][len(test_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]]=1
                            newsA[len(test_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]][el]=1
                            newsA[len(test_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]][len(test_user_his[i][s])+len(category)+entityid_set[user_entity[s][el][e]]]=1
                entityid_set_ids=entityid_set_ids[:50]
                entityid_set_ids+=[0]*(50-len(entityid_set_ids))
        
                for m in range(len(test_user_his[i][s])):
                    if test_user_his[i][s][m]!=0:
                        newsA[m][m]=1
                        if m>=1 and test_user_his[i][s][m-1]!=0:
                            newsA[m][m-1]=1
                        if m<len(test_user_his[i][s])-1 and test_user_his[i][s][m+1]!=0:
                            newsA[m][m+1]=1
                    if user_topic[s][m]!=0:
                        newsA[m][len(test_user_his[i][s])+user_topic[s][m]]=1
                        newsA[len(test_user_his[i][s])+user_topic[s][m]][m]=1
                        newsA[len(test_user_his[i][s])+user_topic[s][m]][len(test_user_his[i][s])+utopic[s][m]]=1
                user_entity_feature.append(entityid_set_ids)
                all_A.append(newsA)
            user_entity_feature=np.array(user_entity_feature)                
    
    
            all_A=np.array(all_A,dtype='float32')
            
            topics=np.array([list(range(len(category)))]*len(i),dtype='int32')
            
            yield ([user_his,user_entity_feature,topics,all_A])
            

In [39]:
results=[]
for ep in range(3):

    traingen=generate_batch_data_random(30)
    
    model.fit_generator(traingen, epochs=1,steps_per_epoch=len(train_label)//30)
    usermodel = Model([review_input,all_entity_input,all_topic_input ,A_input], u_att)

    valgen=generate_batch_data_user(30) 
    uservec=usermodel.predict_generator(valgen,steps=len(test_user_his)//30+1,verbose=1)
    
    newsvec=candidate_encoder.predict([news_words,news_topic,news_entity],batch_size=50,verbose=1)
    
    predictsession=[]
    
    for i in range(len(uservec)):
        newscand=newsvec[test_candidate[test_index[i][0]:test_index[i][1]]]
        uvector=uservec[i]
        scores=1/(1+np.exp(-np.tensordot(uvector,newscand,axes=(0,1))))
        predictsession.append(scores)
        if i%10000==0:
            print(i)
    
    from sklearn.metrics import roc_auc_score
    all_auc=[]
    all_mrr=[]
    all_ndcg=[]
    all_ndcg2=[]
    
    for t in range(len(predictsession)):
        m=test_index[t]
        if m[1]-m[0]>1:
            all_auc.append(roc_auc_score(test_label[m[0]:m[1]],predictsession[t]))
            all_mrr.append(mrr_score(test_label[m[0]:m[1]],predictsession[t]))
            all_ndcg.append(ndcg_score(test_label[m[0]:m[1]],predictsession[t],k=5))
            all_ndcg2.append(ndcg_score(test_label[m[0]:m[1]],predictsession[t],k=10))
            if len(all_auc)%10000==0:
                print(len(all_auc))
    results.append([np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2)])
    print(np.mean(all_auc),np.mean(all_mrr),np.mean(all_ndcg),np.mean(all_ndcg2))
    